# StreetFighter

# Save Model Progress To Video

In [1]:
# from StreetFighter_env import StreetFighter
import retro
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
import os

In [2]:
from gymnasium import Env
from gymnasium.spaces import Box, MultiBinary
import numpy as np
import cv2
import random


In [3]:
VIDEO_FOLDER = "./videos/"
VIDEO_EVERY_N_EPISODES = 1
os.makedirs(VIDEO_FOLDER, exist_ok=True)

In [4]:
class StreetFighter(Env):
    # Constructor now accepts savestates and render_mode
    def __init__(self, savestates=None):
        super().__init__()
        
        # Define observation and action spaces
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        
        
        # Initialize the game instance, passing render_mode to retro.make
        self.game = retro.make(
            game='StreetFighterIISpecialChampionEdition-Genesis',
            use_restricted_actions=retro.Actions.FILTERED,
            record="./videos"
        )
        
        # Store the list of savestates for random initialization
        # Ensure it's always a list, even if None is passed
        self.savestates = savestates if savestates is not None else []
        
        # Initialize attributes that will be used in step and reset
        self.previous_frame = None
        self.score = 0

    # The single, correct reset method
    # It accepts seed and options for Gymnasium API compatibility
    def reset(self, seed=None, options=None):
        # Set seed for reproducibility if provided
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)

        # Randomly choose a savestate if any are provided
        if self.savestates:
            selected_state = random.choice(self.savestates)
            self.game.load_state(selected_state) 
            # Get the screen after loading the state
            obs = self.game.get_screen()
            # print(f"Loaded savestate: {selected_state}")
        else:
            # Fallback to default game reset if no savestates are configured
            obs = self.game.reset() # retro.reset() returns the initial observation
            # print("Resetting to default game start.")

        # Preprocess the initial observation
        obs = self.preprocess(obs)
        self.previous_frame = obs # Store for frame delta calculation

        # Reset score for the new episode
        self.score = 0

        # Gymnasium API requires returning observation and an info dictionary
        return obs, {} # Return obs and an empty info dict

    def preprocess(self, observation):
        # Grayscaling
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize to 84x84
        resize = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_CUBIC)
        # Add a channel dimension (84, 84, 1)
        channels = np.reshape(resize, (84, 84, 1))
        return channels

    # The step method, returning 5 values as per Gymnasium API
    def step(self, action):
        # Take a step in the game
        # retro.step returns obs, reward, done, info (4 values)
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)

        # Calculate frame delta
        # Ensure previous_frame is initialized (should be by reset)
        if self.previous_frame is None: # Fallback, should not happen if reset is called
            self.previous_frame = np.zeros_like(obs)
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs # Update previous frame for next step

        # Reshape the reward based on score delta
        current_score = info['score']
        step_reward = current_score - self.score
        self.score = current_score # Update current score

        # Gymnasium API expects 5 return values: obs, reward, terminated, truncated, info
        # 'done' from retro typically means the episode is terminated (game over)
        terminated = done
        truncated = False # Set to True if episode ends due to time limit or other non-game-over reason

        return frame_delta, step_reward, terminated, truncated, info

    # The render method
    # It will return an rgb_array if render_mode='rgb_array' was set in __init__
    # Otherwise, it will return None (or handle display if render_mode='human')
    def render(self, *args, **kwargs):
        if self.render_mode is not None:
            return self.game.render()
        return None # Return None if no render_mode is set or if it's not 'rgb_array'



    def close(self):
        self.game.close()

In [5]:
model = PPO.load('./train/model_5M.zip')
env = StreetFighter()

c:\Users\james\anaconda3\envs\gym\lib\site-packages\stable_baselines3\common\save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(file_content

In [6]:
# env = Monitor(env) # Keep Monitor for logging, if desired
env = DummyVecEnv([lambda: env])
print("Environment wrapped with Monitor and DummyVecEnv.")


Environment wrapped with Monitor and DummyVecEnv.


In [7]:
num_episodes = 3  # Number of test episodes to run
num_steps_per_episode = 8000

In [8]:
print("Starting testing...")
for episode in range(num_episodes):
    obs = env.reset()
    done = False
    total_reward = 0
    step = 0
    print(f"--- Episode {episode + 1} ---")
    while not done and step < num_steps_per_episode:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        step += 1

print("Testing finished. Closing environment.")
env.close()
print("Environment closed. Video(s) should be in the ./videos/ folder.")


Starting testing...
--- Episode 1 ---
--- Episode 2 ---
--- Episode 3 ---
Testing finished. Closing environment.
Environment closed. Video(s) should be in the ./videos/ folder.


# Playback

In [9]:
env.close()

In [10]:
movie = retro.Movie('./videos/StreetFighterIISpecialChampionEdition-Genesis-Champion.Level1.RyuVsGuile-000000.bk2')
movie.step()

env = retro.make(
    game=movie.get_game(),
    state=None,
    # bk2s can contain any button presses, so allow everything
    use_restricted_actions=retro.Actions.ALL,
    players=movie.players,
)
env.initial_state = movie.get_state()
env.reset()

while movie.step():
    keys = []
    for p in range(movie.players):
        for i in range(env.num_buttons):
            keys.append(movie.get_key(i, p))
    env.step(keys)

In [13]:
#converts playback to MP4
!python -m retro.scripts.playback_movie ./videos/StreetFighterIISpecialChampionEdition-Genesis-Champion.Level1.RyuVsGuile-000005.bk2

^C
